In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model, Sequential
from pyts.image import GramianAngularField
from collections import deque
import random

In [2]:
file_path = '../bar_movement/data/'
currency_pair = 'Eur_Usd'

In [3]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_H4_2015-2023.csv')
df.Date = pd.to_datetime(df.Date, utc=True)
df.reset_index(drop=True, inplace=True)

In [4]:
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth

def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def chop(df, lookback=14):
    atr1 = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

def williams_r(highs, lows, closes, length=21, ema_length=15):
    highest_highs = highs.rolling(window=length).max()
    lowest_lows = lows.rolling(window=length).min()

    willy = 100 * (closes - highest_highs) / (highest_highs - lowest_lows)
    willy_ema = pd.Series.ewm(willy, span=ema_length).mean()

    return willy, willy_ema

def squeeze(barsdata, length=20, length_kc=20, mult=1.5):
    # Bollinger bands
    m_avg = barsdata['Mid_Close'].rolling(window=length).mean()
    m_std = barsdata['Mid_Close'].rolling(window=length).std(ddof=0)
    upper_bb = m_avg + mult * m_std
    lower_bb = m_avg - mult * m_std

    # Keltner channel
    tr0 = abs(barsdata['Mid_High'] - barsdata['Mid_Low'])
    tr1 = abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    tr2 = abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    tr = pd.concat([tr0, tr1, tr2], axis=1).max(axis=1)
    range_ma = tr.rolling(window=length_kc).mean()
    upper_kc = m_avg + range_ma * mult
    lower_kc = m_avg - range_ma * mult

    # Squeeze
    squeeze_on = (lower_bb > lower_kc) & (upper_bb < upper_kc)

    return squeeze_on

def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).mean()

def atr_bands(high, low, close, lookback=14, atr_multiplier=3):
    scaled_atr_vals = atr(high, low, close, lookback) * atr_multiplier
    lower_band = close - scaled_atr_vals
    upper_band = close + scaled_atr_vals

    return lower_band, upper_band

def rsi(closes, periods=14):
    close_delta = closes.diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100 / (1 + rsi))

    return rsi

def qqe_mod(closes, rsi_period=6, smoothing=5, qqe_factor=3, threshold=3, mult=0.35, sma_length=50):
    Rsi = rsi(closes, rsi_period)
    RsiMa = Rsi.ewm(span=smoothing).mean()
    AtrRsi = np.abs(RsiMa.shift(1) - RsiMa)
    Wilders_Period = rsi_period * 2 - 1
    MaAtrRsi = AtrRsi.ewm(span=Wilders_Period).mean()
    dar = MaAtrRsi.ewm(span=Wilders_Period).mean() * qqe_factor

    longband = pd.Series(0.0, index=Rsi.index)
    shortband = pd.Series(0.0, index=Rsi.index)
    trend = pd.Series(0, index=Rsi.index)

    DeltaFastAtrRsi = dar
    RSIndex = RsiMa
    newshortband = RSIndex + DeltaFastAtrRsi
    newlongband = RSIndex - DeltaFastAtrRsi
    longband = pd.Series(np.where((RSIndex.shift(1) > longband.shift(1)) & (RSIndex > longband.shift(1)),
                        np.maximum(longband.shift(1), newlongband), newlongband))
    shortband = pd.Series(np.where((RSIndex.shift(1) < shortband.shift(1)) & (RSIndex < shortband.shift(1)),
                        np.minimum(shortband.shift(1), newshortband), newshortband))
    cross_1 = (longband.shift(1) < RSIndex) & (longband > RSIndex)
    cross_2 = (RSIndex > shortband.shift(1)) & (RSIndex.shift(1) < shortband)
    trend = np.where(cross_2, 1, np.where(cross_1, -1, trend.shift(1).fillna(1)))
    FastAtrRsiTL = pd.Series(np.where(trend == 1, longband, shortband))

    basis = (FastAtrRsiTL - 50).rolling(sma_length).mean()
    dev = mult * (FastAtrRsiTL - 50).rolling(sma_length).std()
    upper = basis + dev
    lower = basis - dev

    Greenbar1 = RsiMa - 50 > threshold
    Greenbar2 = RsiMa - 50 > upper

    Redbar1 = RsiMa - 50 < 0 - threshold
    Redbar2 = RsiMa - 50 < lower

    Greenbar = Greenbar1 & Greenbar2
    Redbar = Redbar1 & Redbar2

    return Greenbar, Redbar, RsiMa - 50

def heikin_ashi(open_values, high_values, low_values, close_values):
    ha_close = (open_values + high_values + low_values + close_values) / 4

    ha_open = pd.Series(0.0, index=open_values.index)
    ha_open.iloc[0] = open_values.iloc[0]

    for i in range(1, len(open_values)):
        ha_open.iloc[i] = (ha_open.iloc[i - 1] + ha_close.iloc[i - 1]) / 2

    ha_high = pd.concat([ha_open, ha_close, high_values], axis=1).max(axis=1)
    ha_low = pd.concat([ha_open, ha_close, low_values], axis=1).min(axis=1)

    return ha_open, ha_high, ha_low, ha_close

def trend_indicator(opens, highs, lows, closes, ema_period=50, smoothing_period=10):
    ha_open, _, _, ha_close = heikin_ashi(opens, highs, lows, closes)

    ha_o_ema = pd.Series.ewm(ha_open, span=ema_period).mean()
    ha_c_ema = pd.Series.ewm(ha_close, span=ema_period).mean()

    ha_o_ema_smooth = pd.Series.ewm(ha_o_ema, span=smoothing_period).mean()
    ha_c_ema_smooth = pd.Series.ewm(ha_c_ema, span=smoothing_period).mean()

    return ha_c_ema_smooth > ha_o_ema_smooth

def supertrend(barsdata, atr_len=10, mult=3):
    curr_atr = atr(barsdata['Mid_High'], barsdata['Mid_Low'], barsdata['Mid_Close'], lookback=atr_len)
    highs, lows = barsdata['Mid_High'], barsdata['Mid_Low']
    hl2 = (highs + lows) / 2
    final_upperband = hl2 + mult * curr_atr
    final_lowerband = hl2 - mult * curr_atr

    # initialize Supertrend column to True
    supertrend = [True] * len(df)

    close = barsdata['Mid_Close']
    
    for i in range(1, len(df.index)):
        curr, prev = i, i - 1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True

        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False

        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]

            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

    return supertrend, final_upperband, final_lowerband

def fractal(lows, highs, window=20):
    assert len(lows) == len(highs)

    fractal_period = 2 * window + 1

    is_support = lows.rolling(fractal_period, center=True).apply(lambda x: x[window] == min(x), raw=True)
    is_resistance = highs.rolling(fractal_period, center=True).apply(lambda x: x[window] == max(x), raw=True)
    
    is_support_indices = pd.Series(is_support.index[is_support == 1.0])
    is_resistance_indices = pd.Series(is_resistance.index[is_resistance == 1.0])

    support_fractal_vals = lows[is_support_indices].reindex(lows.index).ffill()
    resistance_fractal_vals = highs[is_resistance_indices].reindex(highs.index).ffill()

    return support_fractal_vals, resistance_fractal_vals

In [5]:
df['sin_hour'] = np.sin(2 * np.pi * df['Date'].dt.hour / 24)
df['sin_day'] = np.sin(2 * np.pi * df['Date'].dt.day / 7)
df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['ema100'] = pd.Series.ewm(df['Mid_Close'], span=100).mean()
df['atr'] = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['rsi'] = rsi(df['Mid_Close'])
df['rsi_sma'] = df['rsi'].rolling(50).mean()
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['slow_k'], df['slow_d'] = stoch(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['chop'] = chop(df)
df['vo'] = vo(df['Volume'])
df['willy'], df['willy_ema'] = williams_r(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['squeeze'] = squeeze(df)
df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
df['lower_atr_band'], df['upper_atr_band'] = atr_bands(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['qqe_up'], df['qqe_down'], df['qqe_val'] = qqe_mod(df['Mid_Close'])
df['supertrend'], df['supertrend_ub'], df['supertrend_lb'] = supertrend(df)
df['ti_up'] = trend_indicator(df['Mid_Open'], df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
fractal_window = 15
df['support'], df['resistance'] = fractal(df['Mid_Low'], df['Mid_High'], window=fractal_window)
df['support'], df['resistance'] = df['support'].shift(fractal_window), df['resistance'].shift(fractal_window)

df['pips_up'], df['pips_down'] = abs(df['Mid_High'] - df['Mid_Open']), abs(df['Mid_Open'] - df['Mid_Low'])
df['pips_up_ema'], df['pips_down_ema'] = pd.Series.ewm(df['pips_up'], span=200).mean(), pd.Series.ewm(df['pips_down'], span=200).mean()
df['ask_pips_up'], df['ask_pips_down'] = abs(df['Ask_High'] - df['Ask_Open']), abs(df['Ask_Open'] - df['Ask_Low'])
df['ask_pips_up_ema'], df['ask_pips_down_ema'] = pd.Series.ewm(df['ask_pips_up'], span=200).mean(), pd.Series.ewm(df['ask_pips_down'], span=200).mean()
df['bid_pips_up'], df['bid_pips_down'] = abs(df['Bid_High'] - df['Bid_Open']), abs(df['Bid_Open'] - df['Bid_Low'])
df['bid_pips_up_ema'], df['bid_pips_down_ema'] = pd.Series.ewm(df['bid_pips_up'], span=200).mean(), pd.Series.ewm(df['bid_pips_down'], span=200).mean()

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
cutoff_date = '2022-06-15 05:00:00'

df_train = df.loc[df['Date'] <= cutoff_date]
df_train.reset_index(drop=True, inplace=True)

df_test = df.loc[df['Date'] > cutoff_date]
df_test.reset_index(drop=True, inplace=True)

In [7]:
df_train

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,...,pips_up_ema,pips_down_ema,ask_pips_up,ask_pips_down,ask_pips_up_ema,ask_pips_down_ema,bid_pips_up,bid_pips_down,bid_pips_up_ema,bid_pips_down_ema
0,2015-06-29 17:00:00+00:00,1.12123,1.12771,1.12023,1.12349,1.12146,1.12798,1.12039,1.12370,1.12134,...,0.002550,0.002375,0.00652,0.00107,0.002564,0.002397,0.00648,0.00100,0.002565,0.002356
1,2015-06-29 21:00:00+00:00,1.12370,1.12387,1.11965,1.11989,1.12429,1.12441,1.11982,1.12006,1.12400,...,0.002499,0.002414,0.00012,0.00447,0.002514,0.002440,0.00017,0.00405,0.002515,0.002391
2,2015-06-30 01:00:00+00:00,1.11993,1.12022,1.11809,1.11936,1.12007,1.12039,1.11831,1.11952,1.12000,...,0.002454,0.002401,0.00032,0.00176,0.002468,0.002426,0.00029,0.00184,0.002469,0.002380
3,2015-06-30 05:00:00+00:00,1.11938,1.12010,1.11353,1.11401,1.11954,1.12025,1.11369,1.11427,1.11946,...,0.002419,0.002472,0.00071,0.00585,0.002433,0.002495,0.00072,0.00585,0.002434,0.002451
4,2015-06-30 09:00:00+00:00,1.11400,1.12179,1.11330,1.12126,1.11433,1.12195,1.11357,1.12146,1.11416,...,0.002525,0.002436,0.00762,0.00076,0.002537,0.002460,0.00779,0.00070,0.002542,0.002415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,2022-06-14 13:00:00+00:00,1.04480,1.04591,1.04058,1.04195,1.04497,1.04606,1.04076,1.04209,1.04488,...,0.001722,0.001899,0.00109,0.00421,0.001688,0.001944,0.00111,0.00422,0.001771,0.001867
10837,2022-06-14 17:00:00+00:00,1.04194,1.04428,1.03992,1.04160,1.04208,1.04442,1.04005,1.04180,1.04201,...,0.001728,0.001900,0.00234,0.00203,0.001695,0.001945,0.00234,0.00202,0.001776,0.001869
10838,2022-06-14 21:00:00+00:00,1.04064,1.04398,1.04064,1.04339,1.04164,1.04415,1.04148,1.04357,1.04114,...,0.001740,0.001881,0.00251,0.00016,0.001703,0.001927,0.00334,0.00000,0.001792,0.001850
10839,2022-06-15 01:00:00+00:00,1.04334,1.04509,1.04234,1.04260,1.04352,1.04525,1.04250,1.04277,1.04343,...,0.001740,0.001873,0.00173,0.00102,0.001703,0.001918,0.00175,0.00100,0.001791,0.001842


In [8]:
df_test

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,...,pips_up_ema,pips_down_ema,ask_pips_up,ask_pips_down,ask_pips_up_ema,ask_pips_down_ema,bid_pips_up,bid_pips_down,bid_pips_up_ema,bid_pips_down_ema
0,2022-06-15 09:00:00+00:00,1.04929,1.04970,1.04034,1.04141,1.04946,1.04987,1.04050,1.04155,1.04938,...,0.001789,0.001927,0.00041,0.00896,0.001753,0.001971,0.00041,0.00895,0.001840,0.001897
1,2022-06-15 13:00:00+00:00,1.04136,1.04315,1.03794,1.03812,1.04151,1.04329,1.03806,1.03825,1.04144,...,0.001789,0.001942,0.00178,0.00345,0.001753,0.001986,0.00179,0.00342,0.001839,0.001912
2,2022-06-15 17:00:00+00:00,1.03811,1.04690,1.03582,1.04456,1.03826,1.04708,1.03602,1.04477,1.03818,...,0.001859,0.001945,0.00882,0.00224,0.001824,0.001989,0.00879,0.00229,0.001908,0.001916
3,2022-06-15 21:00:00+00:00,1.04394,1.04685,1.04394,1.04521,1.04494,1.04703,1.04434,1.04535,1.04444,...,0.001865,0.001928,0.00209,0.00060,0.001826,0.001975,0.00291,0.00000,0.001918,0.001897
4,2022-06-16 01:00:00+00:00,1.04520,1.04582,1.04304,1.04377,1.04536,1.04598,1.04319,1.04393,1.04528,...,0.001853,0.001930,0.00062,0.00217,0.001814,0.001977,0.00062,0.00216,0.001905,0.001899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,2023-06-14 13:00:00+00:00,1.08303,1.08637,1.08265,1.08617,1.08319,1.08650,1.08279,1.08631,1.08311,...,0.001314,0.001276,0.00331,0.00040,0.001289,0.001313,0.00334,0.00038,0.001354,0.001261
1556,2023-06-14 17:00:00+00:00,1.08619,1.08623,1.08006,1.08308,1.08632,1.08636,1.08021,1.08325,1.08626,...,0.001301,0.001324,0.00004,0.00611,0.001276,0.001361,0.00004,0.00613,0.001341,0.001309
1557,2023-06-14 21:00:00+00:00,1.08270,1.08438,1.08233,1.08375,1.08370,1.08453,1.08311,1.08390,1.08320,...,0.001301,0.001315,0.00083,0.00059,0.001272,0.001353,0.00168,0.00037,0.001344,0.001300
1558,2023-06-15 01:00:00+00:00,1.08374,1.08375,1.08077,1.08108,1.08390,1.08390,1.08092,1.08125,1.08382,...,0.001288,0.001332,0.00000,0.00298,0.001259,0.001370,0.00001,0.00297,0.001331,0.001316


In [9]:
lookback, training_data = 50, []
pips_multiplier = 100 if 'Jpy' in currency_pair else 10000

In [10]:
def _grab_image_data(subset):
    gasf_transformer = GramianAngularField(method='summation')
    gasf_subset = gasf_transformer.transform(subset)

    return gasf_subset
    
def _create_data_point(row):
    sin_hour, sin_day, mid_open, mid_high, mid_low, mid_close, qqe_val, ema200, ema100, lower_atr_band, upper_atr_band, supertrend_ub, supertrend_lb, rsi, rsi_sma, adx, slow_k, slow_d, chop, vo, willy, willy_ema, squeeze, macd, macdsignal, support, resistance, pips_up, pips_down, pips_up_ema, pips_down_ema, ask_open, bid_open, ask_pips_up, ask_pips_down, ask_pips_up_ema, ask_pips_down_ema, bid_pips_up, bid_pips_down, bid_pips_up_ema, bid_pips_down_ema, = row[['sin_hour', 'sin_day', 'Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close', 'qqe_val', 'ema200', 'ema100', 'lower_atr_band', 'upper_atr_band', 'supertrend_ub', 'supertrend_lb', 'rsi', 'rsi_sma', 'adx', 'slow_k', 'slow_d', 'chop', 'vo', 'willy', 'willy_ema', 'squeeze', 'macd', 'macdsignal', 'support', 'resistance', 'pips_up', 'pips_down', 'pips_up_ema', 'pips_down_ema', 'Ask_Open', 'Bid_Open', 'ask_pips_up', 'ask_pips_down', 'ask_pips_up_ema', 'ask_pips_down_ema', 'bid_pips_up', 'bid_pips_down', 'bid_pips_up_ema', 'bid_pips_down_ema']]

    up_trend = ema100 > ema200
    between_emas = ema200 < mid_close < ema100 if up_trend else ema100 < mid_close < ema200
    above_support, below_resistance = mid_close > support, mid_close < resistance
    dist_to_support, dist_to_resistance = abs(mid_close - support) * pips_multiplier, abs(resistance - mid_close) * pips_multiplier
    dist_to_ema_100, dist_to_ema_200 = abs(mid_close - ema100) * pips_multiplier, abs(mid_close - ema200) * pips_multiplier
    dist_between_emas = abs(ema200 - ema100) * pips_multiplier
    dist_between_atr_bands = abs(upper_atr_band - lower_atr_band) * pips_multiplier
    dist_to_upper_atr = (upper_atr_band - mid_high) * pips_multiplier
    dist_to_lower_atr = (mid_low - lower_atr_band) * pips_multiplier
    dist_between_supertrend_bands = abs(supertrend_ub - supertrend_lb) * pips_multiplier
    dist_to_upper_supertrend = (supertrend_ub - mid_high) * pips_multiplier
    dist_to_lower_supertrend = (mid_low - supertrend_lb) * pips_multiplier
    bullish = mid_close > mid_open
    candle_size = abs(mid_high - mid_low)
    if candle_size == 0:
        candle_size = 0.00001 if 'Jpy' in currency_pair else 0.0000001
    body_size_percentage = abs(mid_close - mid_open) / candle_size
    upper_wick_percentage, lower_wick_percentage = abs(min(mid_open, mid_close) - mid_low) / candle_size, abs(mid_high - max(mid_open, mid_close)) / candle_size
    candle_size *= pips_multiplier

    rsi_up = rsi > rsi_sma
    adx_large = adx > 30
    chop_small = chop < 0.5
    vo_positive = vo > 0
    willy_above_ema = willy > willy_ema
    macd_above_signal = macd > macdsignal
    macd_above_zero = min(macd, macdsignal) > 0

    spread = abs(ask_open - bid_open) * pips_multiplier

    # return [sin_hour, sin_day, spread, mid_open, mid_high, mid_low, mid_close, ask_pips_up, ask_pips_down, ask_pips_up_ema, ask_pips_down_ema, bid_pips_up, bid_pips_down, bid_pips_up_ema, bid_pips_down_ema, qqe_val, 
    #         rsi, rsi_up, adx_large, chop_small, vo_positive, willy_above_ema, macd_above_signal, 
    #         macd_above_zero, up_trend, between_emas, above_support, below_resistance, dist_to_support, dist_to_resistance, 
    #         dist_to_ema_100, dist_to_ema_200, dist_between_emas, dist_between_atr_bands, dist_to_upper_atr, dist_to_lower_atr, 
    #         dist_between_supertrend_bands, dist_to_upper_supertrend, dist_to_lower_supertrend, bullish, candle_size, 
    #         body_size_percentage, upper_wick_percentage, lower_wick_percentage]
    return [spread, ask_pips_up * pips_multiplier, ask_pips_down * pips_multiplier, ask_pips_up_ema * pips_multiplier, ask_pips_down_ema * pips_multiplier, bid_pips_up * pips_multiplier, bid_pips_down * pips_multiplier, bid_pips_up_ema * pips_multiplier, bid_pips_down_ema * pips_multiplier,
    qqe_val, rsi, rsi_up, adx, adx_large, chop, chop_small, vo, vo_positive]

In [11]:
for i in range(lookback, len(df_train)):
    mid_open, mid_high, mid_low, ask_open, ask_high, ask_low, bid_open, bid_high, bid_low = df_train.loc[df_train.index[i], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Bid_Open', 'Bid_High', 'Bid_Low']]
    pips_up, pips_down = abs(mid_high - mid_open) * pips_multiplier, abs(mid_open - mid_low) * pips_multiplier

    data_array = []

    for j in range(i - lookback, i):
        data_array.append(_create_data_point(df_train.iloc[j, :]))

    assert len(data_array) == lookback

    seq = _grab_image_data(data_array)

    # training_data.append([seq, np.array([pips_up, pips_down])])
    # training_data.append([seq, np.array([ask_high, ask_low, bid_high, bid_low])])
    ask_pips_up, ask_pips_down, bid_pips_up, bid_pips_down = abs(ask_high - ask_open) * pips_multiplier, abs(ask_open - ask_low) * pips_multiplier, abs(bid_high - bid_open) * pips_multiplier, abs(bid_open - bid_low) * pips_multiplier
    training_data.append([seq, np.array([ask_pips_up, ask_pips_down, bid_pips_up, bid_pips_down])])

In [12]:
np.random.shuffle(training_data)

train_set_ratio = 0.7
cutoff_index = int(len(training_data) * train_set_ratio)
train_set, validation_set = training_data[:cutoff_index], training_data[cutoff_index:]

In [13]:
x_train = []
y_train = []

for seq, target in train_set:
  x_train.append(seq)
  y_train.append(target)

x_validation = []
y_validation = []

for seq, target in validation_set:
  x_validation.append(seq)
  y_validation.append(target)

x_train = np.array(x_train)
y_train = np.array(y_train)
x_validation = np.array(x_validation)
y_validation = np.array(y_validation)

print(x_train.shape, y_train.shape, x_validation.shape, y_validation.shape)

(7553, 50, 18, 18) (7553, 4) (3238, 50, 18, 18) (3238, 4)


In [14]:
input_data_shape = x_train.shape[1:]

model = Sequential()

# model.add(Conv2D(filters = 16, kernel_size = (3,3), padding ='Same', activation ='relu', input_shape = input_data_shape))
# model.add(Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same', activation ='relu'))
# model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation ='relu'))

# model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# # model.add(Dense(128, activation = "relu"))
# # model.add(Dropout(0.5))
# # model.add(Dense(128, activation = "relu"))
# # model.add(Dropout(0.5))
# model.add(Dense(16, activation = "relu"))
# # model.add(Dropout(0.25))
# # model.add(Dense(32, activation = "relu"))
# model.add(Dense(2, activation = "relu"))

# Block 1
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=input_data_shape))
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

# Block 2
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

# Block 3
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

# Output/final block
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))

Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2023-08-23 13:38:04.790984: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-23 13:38:04.791152: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [15]:
# Hyperparameters
n_epochs = 1000
batch_size = 32
optimizer = tf.keras.optimizers.Adam()
model_file_path = f'./cnn_h4_{currency_pair}'

In [16]:
early_stop = EarlyStopping(monitor='val_mean_squared_error', verbose=1, patience=int(n_epochs * 0.1))
model_checkpoint = ModelCheckpoint(model_file_path, monitor='val_mean_squared_error', save_best_only=True, verbose=1)

optimizer = Adam()

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])

In [17]:
history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=n_epochs,
    validation_data=(x_validation, y_validation),
    callbacks=[early_stop, model_checkpoint]
)

Epoch 1/1000


2023-08-23 13:38:05.184608: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-08-23 13:38:05.423640: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


237/237 [==============================] - ETA: 0s - loss: 327.7867 - mean_squared_error: 327.7867

2023-08-23 13:38:09.779978: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_mean_squared_error improved from inf to 325.29919, saving model to ./cnn_h4_Eur_Usd


...

INFO:tensorflow:Assets written to: ./cnn_h4_Eur_Usd/assets


237/237 [==============================] - 5s 20ms/step - loss: 217.6260 - mean_squared_error: 217.6260 - val_loss: 231.3459 - val_mean_squared_error: 231.3459
Epoch 14/1000
237/237 [==============================] - ETA: 0s - loss: 218.4579 - mean_squared_error: 218.4579
Epoch 14: val_mean_squared_error did not improve from 231.34589
237/237 [==============================] - 4s 18ms/step - loss: 218.4579 - mean_squared_error: 218.4579 - val_loss: 242.9279 - val_mean_squared_error: 242.9279
Epoch 15/1000
235/237 [============================>.] - ETA: 0s - loss: 212.9707 - mean_squared_error: 212.9707
Epoch 15: val_mean_squared_error improved from 231.34589 to 230.14458, saving model to ./cnn_h4_Eur_Usd


INFO:tensorflow:Assets written to: ./cnn_h4_Eur_Usd/assets


INFO:tensorflow:Assets written to: ./cnn_h4_Eur_Usd/assets


237/237 [==============================] - 5s 21ms/step - loss: 214.2514 - mean_squared_error: 214.2514 - val_loss: 230.1446 - val_mean_squared_error: 230.1446
Epoch 16/1000
235/237 [============================>.] - ETA: 0s - loss: 204.6013 - mean_squared_error: 204.6013
Epoch 16: val_mean_squared_error did not improve from 230.14458
237/237 [==============================] - 4s 18ms/step - loss: 204.4166 - mean_squared_error: 204.4166 - val_loss: 240.0275 - val_mean_squared_error: 240.0275
Epoch 17/1000
237/237 [==============================] - ETA: 0s - loss: 200.7542 - mean_squared_error: 200.7542
Epoch 17: val_mean_squared_error did not improve from 230.14458
237/237 [==============================] - 4s 18ms/step - loss: 200.7542 - mean_squared_error: 200.7542 - val_loss: 232.9110 - val_mean_squared_error: 232.9110
Epoch 18/1000
234/237 [============================>.] - ETA: 0s - loss: 206.1151 - mean_squared_error: 206.1151
Epoch 18: val_mean_squared_error did not improve from

In [18]:
value_per_pip = 1.0
amounts_per_day = [-0.008, -0.01, -0.012] if 'Jpy' in currency_pair else [-0.00008, -0.0001, -0.00012]
rounding = 3 if 'Jpy' in currency_pair else 5
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_2022-2023.csv')
df.Date = pd.to_datetime(df.Date, utc=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
model = load_model(model_file_path)
validation_avg_error = 250.32816 ** 0.5

In [21]:
def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def run_simulation(sl_multiplier, atr_multiplier, adapt_errors, error_array_len, sd):
    reward, day_fees, n_wins, n_losses, win_streak, loss_streak, curr_win_streak, curr_loss_streak, n_buys, n_sells = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    pips_risked, win_amounts, loss_amounts = [], [], []
    prev_long_date, trade = None, None

    ask_pips_up_errors, ask_pips_down_errors, bid_pips_up_errors, bid_pips_down_errors = deque(maxlen=error_array_len), deque(maxlen=error_array_len), deque(maxlen=error_array_len), deque(maxlen=error_array_len)

    for _ in range(error_array_len):
        ask_pips_up_errors.append(validation_avg_error)
        ask_pips_down_errors.append(validation_avg_error)
        bid_pips_up_errors.append(validation_avg_error)
        bid_pips_down_errors.append(validation_avg_error)

    for i in range(len(df)):
        curr_date, curr_ao, curr_bo, curr_mid_open, curr_ask_low, curr_bid_high, curr_bid_low, curr_ask_high, curr_bid_close, curr_ask_close = df.loc[df.index[i], ['Date', 'Ask_Open', 'Bid_Open', 'Mid_Open', 'Ask_Low', 'Bid_High', 'Bid_Low', 'Ask_High', 'Bid_Close', 'Ask_Close']]
        spread = abs(curr_ao - curr_bo)

        curr_long_df = df_test.loc[df_test.Date <= curr_date]
        gte = df_test.loc[df_test.Date >= curr_date]

        if len(gte) == 0:
            break

        curr_long_df_len = len(curr_long_df)

        if curr_long_df_len < lookback + 1:
            continue

        curr_long_date = curr_long_df.loc[curr_long_df.index[-2], 'Date']

        if trade is None and curr_long_date != prev_long_date:
            prev_long_date = curr_long_date

            mid_opens = list(curr_long_df.loc[curr_long_df.index[-13:-1], 'Mid_Open'])
            mid_closes = list(curr_long_df.loc[curr_long_df.index[-13:-1], 'Mid_Close'])

            def _check_bars(pips_moved, buy):
                for j in range(len(mid_opens) - 1, -1, -1):
                    if (buy and mid_opens[j] < mid_closes[j]) or (not buy and mid_opens[j] > mid_closes[j]):
                        pips_moved += abs(mid_opens[j] - mid_closes[j])

                        if pips_moved >= 0.0030:
                            return True
                        
                    else:
                        return False
                        
                return False

            buy_signal = _check_bars(0, buy=True)
            sell_signal = _check_bars(0, buy=False)

            if buy_signal or sell_signal:
                data_array = []

                for j in range(curr_long_df_len - lookback - 1, curr_long_df_len - 1):
                    data_array.append(_create_data_point(curr_long_df.iloc[j, :]))

                assert len(data_array) == lookback

                seq = _grab_image_data(data_array)

                ask_pips_up_pred, ask_pips_down_pred, bid_pips_up_pred, bid_pips_down_pred = model.predict(seq.reshape(1, lookback, 18, 18))[0]

                ask_pips_up_error_avg = np.array(ask_pips_up_errors).mean() if adapt_errors else validation_avg_error
                ask_pips_down_error_avg = np.array(ask_pips_down_errors).mean() if adapt_errors else validation_avg_error
                bid_pips_up_error_avg = np.array(bid_pips_up_errors).mean() if adapt_errors else validation_avg_error
                bid_pips_down_error_avg = np.array(bid_pips_down_errors).mean() if adapt_errors else validation_avg_error

                ask_pips_up_pred = abs(ask_pips_up_pred + np.random.normal(0.0, sd) * ask_pips_up_error_avg)
                ask_pips_down_pred = abs(ask_pips_down_pred + np.random.normal(0.0, sd) * ask_pips_down_error_avg)
                bid_pips_up_pred = abs(bid_pips_up_pred + np.random.normal(0.0, sd) * bid_pips_up_error_avg)
                bid_pips_down_pred = abs(bid_pips_down_pred + np.random.normal(0.0, sd) * bid_pips_down_error_avg)

                if adapt_errors:
                    ask_open, ask_high, ask_low, bid_open, bid_high, bid_low = curr_long_df.loc[curr_long_df.index[-1], ['Ask_Open', 'Ask_High', 'Ask_Low', 'Bid_Open', 'Bid_High', 'Bid_Low']]
                    ask_pips_up_true, ask_pips_down_true, bid_pips_up_true, bid_pips_down_true = abs(ask_high - ask_open) * pips_multiplier, abs(ask_open - ask_low) * pips_multiplier, abs(bid_high - bid_open) * pips_multiplier, abs(bid_open - bid_low) * pips_multiplier

                    ask_pips_up_errors.append(abs(ask_pips_up_true - ask_pips_up_pred))
                    ask_pips_down_errors.append(abs(ask_pips_down_true - ask_pips_down_pred))
                    bid_pips_up_errors.append(abs(bid_pips_up_true - bid_pips_up_pred))
                    bid_pips_down_errors.append(abs(bid_pips_down_true - bid_pips_down_pred))

                # atr = curr_long_df.loc[curr_long_df.index[-2], 'atr']
                # atr_threshold = atr * pips_multiplier * atr_multiplier

                if buy_signal and max([ask_pips_up_pred, ask_pips_down_pred, bid_pips_up_pred, bid_pips_down_pred]) == bid_pips_up_pred:
                    open_price = float(curr_ao)
                    sl_pips = bid_pips_down_pred / pips_multiplier
                    stop_loss = round(open_price - (sl_pips * sl_multiplier), rounding)

                    if stop_loss < open_price:
                        curr_pips_to_risk = open_price - stop_loss

                        if spread <= curr_pips_to_risk * 0.1:
                            n_units = get_n_units('buy', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                            trade = {'start_index': i, 'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                    'pips_risked': round(curr_pips_to_risk, 5), 'n_units': n_units, 
                                    'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'prev_profit_ratio': None}
                            
                            pips_risked.append(curr_pips_to_risk)
                            n_buys += 1

                elif sell_signal and max([ask_pips_up_pred, ask_pips_down_pred, bid_pips_up_pred, bid_pips_down_pred]) == ask_pips_down_pred:
                    open_price = float(curr_bo)
                    sl_pips = ask_pips_up_pred / pips_multiplier
                    stop_loss = round(open_price + (sl_pips * sl_multiplier), rounding)

                    if stop_loss > open_price:
                        curr_pips_to_risk = stop_loss - open_price

                        if spread <= curr_pips_to_risk * 0.1:
                            n_units = get_n_units('sell', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                            trade = {'start_index': i, 'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                    'pips_risked': round(curr_pips_to_risk, 5), 'n_units': n_units, 
                                    'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'prev_profit_ratio': None}
                            
                            pips_risked.append(curr_pips_to_risk)
                            n_sells += 1

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
                win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
                loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_close > trade['open_price']:
            curr_profit_ratio = (curr_bid_close - trade['open_price']) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # if curr_profit_ratio >= 1.5 and trade['prev_profit_ratio'] == 0.0:
            #     trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * 0.5)
            #     trade['prev_profit_ratio'] = 0.5

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                # while curr_profit_ratio >= trade['prev_profit_ratio'] + 1.5:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    # trade['prev_profit_ratio'] += 0.5
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * trade['prev_profit_ratio'])

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
                win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
                loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_close < trade['open_price']:
            curr_profit_ratio = (trade['open_price'] - curr_ask_close) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # if curr_profit_ratio >= 1.5 and trade['prev_profit_ratio'] == 0.0:
            #     trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * 0.5)
            #     trade['prev_profit_ratio'] = 0.5

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                # while curr_profit_ratio >= trade['prev_profit_ratio'] + 1.5:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    # trade['prev_profit_ratio'] += 0.5
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * trade['prev_profit_ratio'])

    return reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts

In [23]:
sl_multipliers = [0.5, 1.0, 1.5, 2.0] 
# atr_multipliers = [0.0, 0.5, 1.0, 1.5, 2.0] 
atr_multipliers = [0.0]
sds = [0.5, 1.0, 1.5, 2.0]
adapt_errors_vals = [True, False]
error_array_lens = [5, 10, 25, 50] 

all_combos = []

for sl_multiplier in sl_multipliers:
    for atr_multiplier in atr_multipliers:
        for sd in sds:
            for adapt_errors in adapt_errors_vals:
                for err_array_len in error_array_lens:
                    error_array_len = err_array_len if adapt_errors else 1
                    all_combos.append((sl_multiplier, atr_multiplier, adapt_errors, error_array_len, sd))

                    if not adapt_errors:
                        break

best_sl_multiplier, best_atr_multiplier, best_adapt_errors, best_error_array_len, best_sd = None, None, None, None, None
top_n_results, best_rewards, best_reward, runs_finished = 10, [], -np.inf, 0

percentage_to_try = 1.0
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

for sl_multiplier, atr_multiplier, adapt_errors, error_array_len, sd in combos_to_try:
    reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts = run_simulation(sl_multiplier, atr_multiplier, adapt_errors, error_array_len, sd)
    runs_finished += 1

    print(reward, day_fees, reward + day_fees)
    print('Num buys: ' + str(n_buys))
    print('Num sells: ' + str(n_sells))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    if len(win_amounts) > 0:
        print('Avg win amount: ' + str(np.array(win_amounts).mean()))
        print('Min win amount: ' +  str(min(win_amounts)))
        print('Max win amount: ' + str(max(win_amounts)))
    if len(loss_amounts) > 0:
        print('Avg loss amount: ' + str(np.array(loss_amounts).mean()))
        print('Min loss amount: ' +  str(min(loss_amounts)))
        print('Max loss amount: ' + str(max(loss_amounts)))

    print('Remaining runs: ' + str(n_runs - runs_finished))

    total_profit = reward + day_fees

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or total_profit > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(total_profit), 'sl_multiplier': sl_multiplier, 'atr_multiplier': atr_multiplier, 'adapt_errors': adapt_errors, 'error_array_len': error_array_len, 'sd': sd})

    if total_profit > best_reward:
        best_reward = total_profit
        best_sl_multiplier, best_atr_multiplier, best_adapt_errors, best_error_array_len, best_sd = sl_multiplier, atr_multiplier, adapt_errors, error_array_len, sd

    print('Best reward so far: ' + str(best_reward))
    print()


Num runs: 80

1/1 [==============================] - 0s 11ms/step
-499.9839200000033 -94.56518 -594.5491000000034
Num buys: 36
Num sells: 29
Num trades: 65
Num wins: 17
Num losses: 38
Win streak: 4
Loss streak: 9
Avg pips risked: 0.0042147692307692355
Avg win amount: 82.34856529411749
Min win amount: 49.99536000000062
Max win amount: 199.97891999999933
Avg loss amount: -39.58144854166668
Min loss amount: -49.999949999998606
Max loss amount: 0.0
Remaining runs: 79
Best reward so far: -594.5491000000034

1/1 [==============================] - 0s 12ms/step
150.01400999999672 -117.57838 32.43562999999672
Num buys: 37
Num sells: 41
Num trades: 78
Num wins: 22
Num losses: 44
Win streak: 4
Loss streak: 13
Avg pips risked: 0.004065897435897444
Avg win amount: 106.8145695454546
Min win amount: 49.9950500000008
Max win amount: 199.99547999999925
Avg loss amount: -39.284045000000084
Min loss amount: -49.999979999996235
Max loss amount: 0.0
Remaining runs: 78
Best reward so far: 32.43562999999672


KeyboardInterrupt: 

In [24]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best adapt errors val: ' + str(best_adapt_errors))
print('Best error array length: ' + str(best_error_array_len))
print('Best best standard deviation: ' + str(best_sd))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 466.28720000000834
Best sl multiplier: 2.0
Best atr multiplier: 0.0
Best adapt errors val: False
Best error array length: 1
Best best standard deviation: 0.5
-----------------------
Top results:
{'reward': 32, 'sl_multiplier': 2.0, 'atr_multiplier': 0.0, 'adapt_errors': False, 'error_array_len': 1, 'sd': 0.5}
{'reward': 466, 'sl_multiplier': 2.0, 'atr_multiplier': 0.0, 'adapt_errors': False, 'error_array_len': 1, 'sd': 0.5}
{'reward': 183, 'sl_multiplier': 0.5, 'atr_multiplier': 0.0, 'adapt_errors': False, 'error_array_len': 1, 'sd': 1.5}
{'reward': 49, 'sl_multiplier': 0.5, 'atr_multiplier': 0.0, 'adapt_errors': False, 'error_array_len': 1, 'sd': 0.5}
{'reward': 81, 'sl_multiplier': 0.5, 'atr_multiplier': 0.0, 'adapt_errors': False, 'error_array_len': 1, 'sd': 1.0}
{'reward': 101, 'sl_multiplier': 2.0, 'atr_multiplier': 0.0, 'adapt_errors': False, 'error_array_len': 1, 'sd': 1.5}
{'reward': 297, 'sl_multiplier': 0.5, 'atr_multiplier